# Setup

On google colab, you have to restart runtime after running the following line

In [1]:
!pip install omegaconf

In [2]:
from google.colab import drive
drive.mount("/content/drive/")
#"/content/drive/My Drive/NN-kNN/"
folder_name = "/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/"
import sys
sys.path.insert(0,folder_name)

Mounted at /content/drive/


In [3]:
# folder_name = os.getcwd()

In [4]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf

from dataset import cls_small_data as Cdata
import model.cls_model as Cmodel
from dataset import cls_medium_data

from dataset import reg_data as Rdata
import model.reg_model as Rmodel

In [5]:
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# NCA and LMNN setup

In [7]:
pip install metric-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.3 MB/s eta 0:00:00


In [8]:
import metric_learn
from metric_learn import LMNN,NCA

# Data Sets

Supported small dataset for classification:  
'zebra',
'zebra_special',
'bal',
'digits',
'iris',
'wine',
'breast_cancer',
'psych_depression_physical_symptons',
'covid_anxious'

for regression:
'califonia_housing',
'abalone',
'diabets',
'body_fat',
'ziweifaces'

In [9]:
dataset_name = 'covid_anxious'
cfg = conf_file['dataset'][dataset_name]

if dataset_name in ['covid_anxious','psych_depression_physical_symptons',
                    'zebra','zebra_special','bal','digits','iris','wine','breast_cancer']:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = Cdata.Cls_small_data(dataset_name)
elif dataset_name in []:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = cls_medium_data.Cls_medium_data(dataset_name)
else:
    criterion = torch.nn.MSELoss()
    Xs, ys = Rdata.Reg_data(dataset_name)

Columns in the dataset: Index(['SU_ID', 'P_PANEL', 'NATIONAL_WEIGHT', 'REGION_WEIGHT',
       'NATIONAL_WEIGHT_POP', 'REGION_WEIGHT_POP', 'NAT_WGT_COMB_POP',
       'REG_WGT_COMB_POP', 'P_GEO', 'SOC1',
       ...
       'REGION9', 'P_DENSE', 'MODE', 'LANGUAGE', 'MAIL50', 'RACE1_BANNER',
       'RACE2_BANNER', 'INC_BANNER', 'AGE_BANNER', 'HH_BANNER'],
      dtype='object', length=177)


In [10]:
# This section is used to reload the imported module.
# For example, if you made any changes in the model.cls_model, you should run importlib.reload(Cmodel) as long as you set import model.cls_model as Cmodel.
import importlib
importlib.reload(Cdata)

<module 'dataset.cls_small_data' from '/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/cls_small_data.py'>

# Classification with NNKNN

In [11]:
# prompt: make Xs and ys shuffle and then 1/10 small

np.random.seed(0)
idx = np.random.permutation(len(Xs))
Xs = Xs[idx]
ys = ys[idx]

num_train = int(len(Xs) * 0.1)

Xs = Xs[:num_train]
ys = ys[:num_train]



In [12]:
def train_cls(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)

  # Train model
  model = Cmodel.NN_k_NN(X_train,
                         y_train,
                         cfg.ca_weight_sharing,
                         cfg.top_case_enabled,
                         cfg.top_k,
                         cfg.discount,
                         device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    epoch_msg = True

    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, output, predicted_class = model(X_train_batch)
      loss = criterion(output, y_train_batch)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

        epoch_msg = False
      # print("evaluating")
    model.eval()
    with torch.no_grad():
      _, _, output, predicted_class = model(X_test)

      # Calculate accuracy
      accuracy_temp = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)

    elif accuracy_temp > best_accuracy:
      #memorize best model
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0

    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  return best_accuracy, model

In [13]:
accuracies = []
knn_accuracies = []
lmnn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/classifier_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle=True, random_state = None)
enable_lmnn = True

for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  if(enable_lmnn):
    # https://contrib.scikit-learn.org/metric-learn/supervised.html#lmnn
    lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
    ##TODO, change here if you need to use a different one
    # lmnn = metric_learn.MLKR()
    # lmnn = metric_learn.NCA(max_iter=1000)
    lmnn.fit(X_train,y_train)
    knn = KNeighborsClassifier(n_neighbors=5,metric=lmnn.get_metric())
    knn.fit(X_train,y_train)
    # klmnn_accuracies.append( accuracy_score(knn.predict(X_test), y_test))
    lmnn_acc = accuracy_score(knn.predict(X_test), y_test)
    lmnn_accuracies.append(lmnn_acc)

  knn =  KNeighborsClassifier(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_acc  = accuracy_score(knn.predict(X_test), y_test)
  knn_accuracies.append(knn_acc)

  best_accuracy, model = train_cls(X_train,y_train, X_test, y_test, cfg)
  accuracies.append(best_accuracy)

print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")


Epoch [2/1000], Loss: 1.3256
Epoch [4/1000], Loss: 1.0737
Epoch [6/1000], Loss: 1.0599
Epoch [8/1000], Loss: 1.1709
Epoch [10/1000], Loss: 1.0626
Epoch [12/1000], Loss: 1.0337
Epoch [14/1000], Loss: 1.0121
Epoch [16/1000], Loss: 0.8954
Epoch [18/1000], Loss: 0.9008
Epoch [20/1000], Loss: 1.2167
Epoch [22/1000], Loss: 0.9923
Epoch [24/1000], Loss: 0.9151
Epoch [26/1000], Loss: 1.0473
Epoch [28/1000], Loss: 1.0318
Epoch [30/1000], Loss: 0.9307
Epoch [32/1000], Loss: 1.0266
Epoch [34/1000], Loss: 0.9720
Epoch [36/1000], Loss: 1.0387
Epoch [38/1000], Loss: 0.9756
Epoch [40/1000], Loss: 1.0257
Epoch [42/1000], Loss: 1.0014
Epoch [44/1000], Loss: 1.0324
Epoch [46/1000], Loss: 1.0975
patience exceeded, loading best model
Epoch [2/1000], Loss: 1.2684
Epoch [4/1000], Loss: 1.1304
Epoch [6/1000], Loss: 1.0312
Epoch [8/1000], Loss: 1.1399
Epoch [10/1000], Loss: 1.0013
Epoch [12/1000], Loss: 1.0474
Epoch [14/1000], Loss: 1.0316
Epoch [16/1000], Loss: 0.9806
Epoch [18/1000], Loss: 0.9505
Epoch [20/

In [14]:
# prompt: get the unique y values and their counts

unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")


Unique values: [0 1 2 3]
Counts: [1566  538  234  177]


# Regression with NNKNN

In [ ]:
def train_reg(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=cfg.batch_size, shuffle=False)


    # Train model
  model = Rmodel.NN_k_NN_regression(X_train,
                                    y_train,
                                    cfg.ca_weight_sharing,
                                    cfg.top_case_enabled,
                                    cfg.top_k,
                                    cfg.discount,
                                    cfg.class_weight_sharing,
                                    device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    # break # no training
    epoch_msg = True
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, _, predicted_number = model(X_train_batch)
      # break
      loss = criterion(predicted_number.squeeze(), y_train_batch)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
      predicted_numbers = []
      for X_test_batch, _ in test_loader:
        X_test_batch = X_test_batch.to(device)
        _, _, _, predicted_number = model(X_test_batch)
        predicted_numbers.extend(predicted_number.squeeze().cpu().detach())

      predicted_numbers = torch.Tensor(predicted_numbers)
      accuracy_temp = criterion(y_test, predicted_numbers)

    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)
    elif accuracy_temp < best_accuracy:
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0
    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  _, case_activations, _, predicted_number = model(X_test)

  top_case_indices = torch.topk(case_activations, 5, dim=1)[1].cpu()

  accuracy = criterion(y_test, predicted_number.squeeze().cpu())
  y_train = y_train.cpu()
  top_k_average_accuracy = mean_squared_error(torch.mean(y_train[top_case_indices], dim=1), y_test)

  return best_accuracy, accuracy, top_k_average_accuracy, model

In [ ]:
best_accuracies = []
accuracies = []
top_k_average_accuracies = []
knn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/regression_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle = True,random_state = None)


for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]

  knn = KNeighborsRegressor(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_accuracies.append(mean_squared_error(knn.predict(X_test), y_test))

  best_accuracy, accuracy, top_k_average_accuracy, model= train_reg(X_train, y_train, X_test, y_test, cfg)
  best_accuracies.append(best_accuracy)
  accuracies.append(accuracy)
  top_k_average_accuracies.append(top_k_average_accuracy)

print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

# Results Interpretation

In [1]:
def print_model_features(input_model):
  for n, p in model.named_parameters():
    print(n)
    print(p.data)

In [2]:
print_model_features(model)

NameError: name 'model' is not defined

In [ ]:
# for regression only. for classification is different
#feature_activations, case_activations, predicted_number
model.eval()
feature_activations, case_activations, output, predicted_class = model(X_test)

In [ ]:
#inspecting the case activations
top_case_indices = torch.topk(case_activations, 5, dim=1)[1]

In [ ]:
X_test[0]

In [ ]:
X_train[top_case_indices[0][0]]

By comparing the following two blocks' outputs, you can see we are retrieving a good neighbor.

In [ ]:
#sum abs of X_test[0] and the top activated case
sum(abs(X_test[0] - X_train[top_case_indices[0][0]]))

In [ ]:
# prompt: average sum abs of X_test[0] and X_train data
print(np.mean([sum(abs(X_test[0] - X_train[i])) for i in range(len(X_train))]))

TODO:: A better way is to show the distribution of ``X_test[0] - X_train[i]``

In [ ]:
y_train[top_case_indices[0]]

In [ ]:
knn.predict(X_test)[0]

In [ ]:
indices = knn.kneighbors(X_test)[1][0]

In [ ]:
y_train[indices]